In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
from aml_storage import Labels, Block, Descriptor
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product

In [3]:
ham = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)
# MUST CONVERT FROM PYSCF L=1 FORMAT

In [4]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":")

In [5]:
orbs = json.loads(json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r")))

In [6]:
orbs

{'O': [[1, 0, 0],
  [2, 0, 0],
  [3, 0, 0],
  [2, 1, 1],
  [2, 1, -1],
  [2, 1, 0],
  [3, 1, 1],
  [3, 1, -1],
  [3, 1, 0],
  [3, 2, -2],
  [3, 2, -1],
  [3, 2, 0],
  [3, 2, 1],
  [3, 2, 2]],
 'H': [[1, 0, 0], [2, 0, 0], [2, 1, 1], [2, 1, -1], [2, 1, 0]]}

In [7]:
list(product([1,2,3],[1,2]))

[(1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2)]

In [8]:
ki, kj = 0, 0
ham_builder = DescriptorBuilder(["a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j"], ["m1", "m2"], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for i, fi in enumerate(frames[0].symbols):
    ai = zdic[fi]
    for ni, li, mi in orbs[fi]:
        kj = 0
        if mi != -li:
            continue
        for j, fj in enumerate(frames[0].symbols):
            if i>j:
                continue
            aj = zdic[fj]
            print(fi)        
            for nj, lj, mj in orbs[fj]:                
                if mj != -lj:
                    continue
                block_idx = (ai, ni, li, aj, nj, lj)
                if block_idx not in ham_builder.blocks:
                    block = ham_builder.add_block(sparse=block_idx, features=np.asarray([[0]], dtype=np.int32), 
                        components=np.asarray(list(product(range(-li,li+1), range(-lj,lj+1)) ), dtype=np.int32 ) )
                else:
                    block = ham_builder.blocks[block_idx]
                print(block_idx, mi, mj)
                block.add_samples( labels=[( 0, i, j )], data=np.asarray(ham[0][ki:ki+2*li+1, kj:kj+2*lj+1]).reshape(1, -1, 1) )
                kj += 2*lj+1
        ki += 2*li+1
ham_et = ham_builder.build()        






O
(8, 1, 0, 8, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 8, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 8, 3, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 8, 2, 1) 0 -1
wtf (1, 3, 1) 3
(8, 1, 0, 8, 3, 1) 0 -1
wtf (1, 3, 1) 3
(8, 1, 0, 8, 3, 2) 0 -2
wtf (1, 5, 1) 3
O
(8, 1, 0, 1, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 1, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 1, 2, 1) 0 -1
wtf (1, 3, 1) 3
O
(8, 1, 0, 1, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 1, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 1, 0, 1, 2, 1) 0 -1
wtf (1, 3, 1) 3
O
(8, 2, 0, 8, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 8, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 8, 3, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 8, 2, 1) 0 -1
wtf (1, 3, 1) 3
(8, 2, 0, 8, 3, 1) 0 -1
wtf (1, 3, 1) 3
(8, 2, 0, 8, 3, 2) 0 -2
wtf (1, 5, 1) 3
O
(8, 2, 0, 1, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 1, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 1, 2, 1) 0 -1
wtf (1, 3, 1) 3
O
(8, 2, 0, 1, 1, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 1, 2, 0) 0 0
wtf (1, 1, 1) 3
(8, 2, 0, 1, 2, 1) 0 -1
wtf (1, 3, 1) 3
O
(8, 3, 0, 8, 1, 0) 0 0
wtf (1, 1, 1) 3
(

In [17]:
ham_et.sparse["a1"]

Labels([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       dtype=int32)

In [13]:
ham_et.block(0).values

array([[[-20.534034434156133]]], dtype=object)